In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import random
import math

from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold, cross_val_score

In [124]:
def NormalEquation(matX,vecY):
    X_transpose = np.transpose(matX)

    left_side = np.dot(X_transpose,matX)
    right_side = np.dot(X_transpose,vecY)

    ans = np.linalg.inv(left_side).dot(right_side)
    return ans

def CalculateMSE(x_positions, y_positions, w_list):
    n = len(x_positions)
    x_positions = np.column_stack((np.ones(len(x_positions)), x_positions))
    MSE = 0
    for i in range(0, n):
        wx = np.dot(x_positions[i],np.flip(w_list))
        MSE += (wx - y_positions[i]) ** 2
    MSE *= 1/(n)
    # print("new",MSE)
    return MSE

def testCalculateMSE(x_positions, y_positions, w0, w1):
    n = len(x_positions)
    MSE = 0
    for i in range(0, n):
        MSE += (w0 + w1*x_positions[i] - y_positions[i]) ** 2
    MSE *= 1/(n)
    # print("old",MSE)
    return MSE

In [ ]:
df = pd.read_csv("data/sin_noiseless_10sample.csv")
print(df["y"])

In [ ]:
k_folds = KFold(n_splits=5)

for train_index, validate_index in k_folds.split(df[["x", "x^2", "x^3"]]):
    print(train_index, validate_index)
    X_train , X_validate = df[["x", "x^2", "x^3"]].iloc[train_index],df[["x", "x^2", "x^3"]].iloc[validate_index]
    y_train , y_validate = df["y"].iloc[train_index] , df["y"].iloc[validate_index]

    new_X = [[1,1,1,1,1,1,1,1]]
    for i in np.array(X_train).transpose():
        new_X.append(list(i))
    print(new_X)
    X_matrix = np.array(new_X).transpose()
    print(X_matrix)
    print(NormalEquation(X_matrix, y_train))
    print(df[["x", "x^2", "x^3"]].shape)

In [ ]:
def old_cross_validation(X, y, folds):
    #Not done
    k_folds = KFold(n_splits=folds)

    avg_rmse = []

    for train_index, validate_index in k_folds.split(X):
        #print(train_index, validate_index)
        X_train , X_validate = X.iloc[train_index], X.iloc[validate_index]
        y_train , y_validate = y.iloc[train_index] , y.iloc[validate_index]

        new_X = [[1 for i in range(X_train.shape[0])]]
        for i in np.array(X_train).transpose():
            new_X.append(list(i))
        #print(new_X)
        X_matrix = np.array(new_X).transpose()
        model = NormalEquation(X_matrix, y_train)
        #print(X_matrix)
        #print(model)
        avg_rmse.append(CalculateMSE(np.array(X_validate), np.array(y_validate), model)**(1/2))
        #print(testCalculateMSE(np.array(X_validate), np.array(y_validate), model[0], model[1]))
    print(avg_rmse)
    return np.mean(avg_rmse)

In [120]:
def cross_validation(X, y, folds):
    #Not done
    k_folds = KFold(n_splits=folds)
    
    avg_rmse = []
    for train_index, validate_index in k_folds.split(X):
        # print(train_index,validate_index)
        X_train , X_validate = X.iloc[train_index], X.iloc[validate_index]
        y_train , y_validate = y.iloc[train_index] , y.iloc[validate_index]

        result = np.polyfit(X_train.values.flatten(), np.array(y_train), 1)

        avg_rmse.append(CalculateMSE(X_validate.values.flatten(), np.array(y_validate), result))

    return np.mean(avg_rmse)

In [126]:
cross_validation(df[["x"]], df["y"], 5)

1 [1.5203776901144326, 0.6772205605640664, 0.17570218679865976, 0.33243526727846284, 0.07642724175921889]
2 [1.5203776901144326, 0.6772205605640664, 0.17570218679865976, 0.33243526727846284, 0.07642724175921889]
True


0.556432589302968